<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M2-training-word-vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training customized word embeddings

Word embeddings became big around 2013 and are linked to [this paper](https://arxiv.org/abs/1301.3781) with the beautiful title 
*Efficient Estimation of Word Representations in Vector Space* by Tomas Mokolov et al. coming out of Google. This was the foundation of Word2Vec.

The idea behind it is easiest summarized by the following quote: 


> *You shall know a word by the company it keeps (Firth, J. R. 1957:11)*

![](https://ruder.io/content/images/size/w2000/2016/04/word_embeddings_colah.png)

Let me start with a fascinating example of word embeddings in practice. Below, you can see a figure from the paper: 
*Dynamic Word Embeddings for Evolving Semantic Discovery*. Here (in simple terms) the researchers estimated word vectors for from textual inputs in different time-frames. They picked out some terms and person that obviously changed *their company* over the years. Then they look at the relative position of these terms compared to terms that did not change much (anchors). If you are interested in this kind of research, check out [this blog](https://blog.acolyer.org/2018/02/22/dynamic-word-embeddings-for-evolving-semantic-discovery/) that describes the paper briefly or the [original paper](https://arxiv.org/abs/1703.00607).

![alt text](https://adriancolyer.files.wordpress.com/2018/02/evolving-word-embeddings-fig-1.jpeg)

Word embeddings allow us to create term representations that "learn" meaning from semantic and syntactic features. These models take a sequence of sentences as an input and scan for all individual terms that appear in the whole corpus and all their occurrences. Such contextual learning seems to be able to pick up non-trivial conceptual details and it is this class of models that today enable technologies such as chatbots, machine translation and much more.

The early word embedding models were Word2Vec and [GloVe](https://nlp.stanford.edu/projects/glove/).
In December 2017 Facebook presented [fastText](https://fasttext.cc/) (by the way - by 2017 Tomas Mikolov was working for Facebook and is one of the authors of the [paper](https://arxiv.org/abs/1607.04606) that introduces the research behind fastText). This model extends the idea of Word2Vec, enriching these vectors by information from sub-word elements. What does that mean? Words are not only defined by surrounding words but in addition also by the various syllables that make up the word. Why should that be a good idea? Well, now words such as *apple* and *apples* do not only get similar vectors due to them often sharing context but also because they are composed of the same sub-word elements. This comes in particularly handy when we are dealing with language that have a rich morphology such as Turkish or Russian.  This is also great when working with web-text, which is often messy and misspelt.

The current state-of-the-art transformer models go even further and implement context-specificity (a word may change meaning depending on the context in which it occurs)

Now the good news: You will find pre-trained vectors from all mentioned models online. They will do great in most cases. However, when working with specific tasks: Some obscure languages and/or specific technical jargon (specific scientific field or industry e.g. finance, insurance), it is nice to know how to train such word-vectors.


In this tutorial we will train the "classic" Word2Vec model, considering bi-grams. We will also look a bit into data-engineering issues in sequence-training. Finally, we will look at how we can use such models for text representation beyond individual words.

## Data

The data used here are 10k cooking related posts from Reddit. They come in JSON-lines format and can be either downloaded first or opened via requests.

## Plan of attack
In this tutorial we will not be using Spacy, as it is not fast enough for use in training of large language models.
The intent is to understand training from disk - where the file is not opened (with e.g. pandas) and an object in memory but streamed from disk.

In [1]:
# download data (optional when training from memory)
!wget https://raw.githubusercontent.com/aaubs/ds-master/main/data/reddit_r_cooking_sample.jsonl

--2022-11-01 15:09:55--  https://raw.githubusercontent.com/aaubs/ds-master/main/data/reddit_r_cooking_sample.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2675456 (2.6M) [text/plain]
Saving to: ‘reddit_r_cooking_sample.jsonl’

reddit_r_cooking_sa 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2022-11-01 15:09:56 (169 MB/s) - ‘reddit_r_cooking_sample.jsonl’ saved [2675456/2675456]



In [2]:
# installs
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import pandas as pd
import numpy as np
import json

# we will use nltk for sentence tokenization
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# we will be using gensim for training
import gensim
from gensim import utils
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS


# Logging settings
import logging

for handler in logging.root.handlers[:]:
   logging.root.removeHandler(handler)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Simple In-memory training

To better understand the training itself we start with simple model training out of memory. All the data will be loaded with pandas.
Preprocessing results will also be stored in the dataframe. This is a viable approache up a certain data-size. When going beyond 5M texts (depending on the hardware) that's probably not a good idea..

In [4]:
# load data
data = pd.read_json('https://raw.githubusercontent.com/aaubs/ds-master/main/data/reddit_r_cooking_sample.jsonl', lines=True)

In [5]:
data.head()

,text,meta
0,Where do you get the mock duck? I've only rece...,"{'section': 'Cooking', 'utc': '1364690064'}"
1,Microwaves are terrible. Everyone in this sub ...,"{'section': 'Cooking', 'utc': '1368260826'}"
2,My Pro 500 is going on 18 years old. Thing is ...,"{'section': 'Cooking', 'utc': 1518485096}"
3,deglazing works ok. but not as well as on a st...,"{'section': 'Cooking', 'utc': '1413146528'}"
4,Does Google not exist in Germany? 7g dry is 1....,"{'section': 'Cooking', 'utc': 1522171636}"


Word2Vec uses sentences to train, not paragraphs. Therefore we will need to sentence-tokenize.

In [6]:
# NLTK tokenizer:
sent_tokenize('this is a sentence. also that one.')

['this is a sentence.', 'also that one.']

In [7]:
# Let's apply that to all texts
sentences = []
for i in data['text']:
  sentences.extend(sent_tokenize(i))

In [8]:
len(sentences)

29445

Gensim has efficient simple preprocessing as part of the utility functions. That works well for most latin-letter texts. Check out [Gensim docos](https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html) for more into.

In [9]:
# simple prepro (tokenization, lowercase, de-accent (otional))
sentences_prepro = [utils.simple_preprocess(line) for line in sentences]

We are not removing stopwords for Word2Vec, as the model actually cares about syntax. One thing that we can do is identifying n-grams (phrases).

In [11]:
# trainig a model to identify n-grams
phrase_model = Phrases(sentences_prepro, min_count=1, threshold=1, connector_words=ENGLISH_CONNECTOR_WORDS)

2022-11-01 15:21:05,312 : INFO : collecting all words and their counts
2022-11-01 15:21:05,332 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-11-01 15:21:05,700 : INFO : PROGRESS: at sentence #10000, processed 123637 words and 74036 word types
2022-11-01 15:21:05,930 : INFO : PROGRESS: at sentence #20000, processed 245368 words and 129436 word types
2022-11-01 15:21:06,109 : INFO : collected 176735 token types (unigram + bigrams) from a corpus of 359868 words and 29445 sentences
2022-11-01 15:21:06,110 : INFO : merged Phrases<176735 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2022-11-01 15:21:06,115 : INFO : Phrases lifecycle event {'msg': 'built Phrases<176735 vocab, min_count=1, threshold=1, max_vocab_size=40000000> in 0.80s', 'datetime': '2022-11-01T15:21:06.115308', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [13]:
# apply the model
sentences_phrased = [phrase_model[line] for line in sentences_prepro]

In [14]:
# quick check
sentences_phrased[:5]

[['where_do', 'you_get', 'the', 'mock', 'duck'],
 ['ve_only', 'recently', 'tried_it', 'in', 'restaurant', 'and', 'loved_it'],
 ['hoisin', 'we_use', 'for', 'sandwich', 'condiment', 'mixed_with_sriracha'],
 ['you_could', 'make_those', 'pancakes', 'with', 'another', 'faux', 'meat'],
 ['some_of_those',
  'grain',
  'sausages_are',
  'really_good',
  'and',
  'you_can',
  'slice_them']]

obviousely, some hyperparameter tuning is needed

In [15]:
# adjusting min_count and threshold (that's a value calculated within the model - read docus)
phrase_model = Phrases(sentences_prepro, min_count=25, threshold=20, connector_words=ENGLISH_CONNECTOR_WORDS)
sentences_phrased = [phrase_model[line] for line in sentences_prepro]
sentences_phrased[:5]

2022-11-01 15:23:04,636 : INFO : collecting all words and their counts
2022-11-01 15:23:04,641 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-11-01 15:23:04,806 : INFO : PROGRESS: at sentence #10000, processed 123637 words and 74036 word types
2022-11-01 15:23:04,969 : INFO : PROGRESS: at sentence #20000, processed 245368 words and 129436 word types
2022-11-01 15:23:05,155 : INFO : collected 176735 token types (unigram + bigrams) from a corpus of 359868 words and 29445 sentences
2022-11-01 15:23:05,158 : INFO : merged Phrases<176735 vocab, min_count=25, threshold=20, max_vocab_size=40000000>
2022-11-01 15:23:05,162 : INFO : Phrases lifecycle event {'msg': 'built Phrases<176735 vocab, min_count=25, threshold=20, max_vocab_size=40000000> in 0.53s', 'datetime': '2022-11-01T15:23:05.162199', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


[['where', 'do', 'you', 'get', 'the', 'mock', 'duck'],
 ['ve',
  'only',
  'recently',
  'tried',
  'it',
  'in',
  'restaurant',
  'and',
  'loved',
  'it'],
 ['hoisin',
  'we',
  'use',
  'for',
  'sandwich',
  'condiment',
  'mixed',
  'with',
  'sriracha'],
 ['you',
  'could',
  'make',
  'those',
  'pancakes',
  'with',
  'another',
  'faux',
  'meat'],
 ['some',
  'of',
  'those',
  'grain',
  'sausages',
  'are',
  'really',
  'good',
  'and',
  'you',
  'can',
  'slice',
  'them']]

In [16]:
# did we actually find anything?
for phrase, score in phrase_model.find_phrases(sentences_prepro).items():
    print(phrase, score)

as_well 27.471339649272448
ve_been 41.653628014475565
stainless_steel 339.0599520383693
your_own 20.223897445413495
more_than 20.47408324458768
stir_fry 192.49911686782454
salt_pepper 31.312819683243973
olive_oil 184.98806397708285
store_bought 37.18857840249137
sour_cream 170.60931322975085
ve_never 32.897408361970214
slow_cooker 332.58133391235117
mashed_potatoes 166.10432330827066
thank_you 20.47161354330867
tomato_sauce 20.939855748581923
they_re 27.604060913705585
ve_got 21.553749715437903
check_out 30.309552392385527
talking_about 51.24875724937863
cast_iron 782.157477411027
alton_brown 256.58036640165915
pulled_pork 112.0196486780152
http_www 292.3240096923725
com_recipes 21.29508394248534
better_than 33.40834415963816
don_know 24.0042240154292
sous_vide 1497.1500605082697
next_time 35.865252904469585
grocery_store 213.3450024142926
imgur_com 81.90843485169492
ground_beef 91.19453044375645
grew_up 45.984822202948834
make_sure 24.00867902694272
chicken_breasts 29.188962816157037


Once sentences are pre-processed (tokenized, list of lists) we can train the model.

In [ ]:
model = Word2Vec(sentences=sentences_phrased, 
                               vector_size=300, 
                               window=5, 
                               min_count=5, 
                               workers=4, 
                               epochs=15)

In [30]:
# check most similar terms
model.wv.most_similar('chinese')

[('indian', 0.8582229614257812),
 ('mexican', 0.8339536786079407),
 ('korean', 0.8001686334609985),
 ('asian', 0.7954041957855225),
 ('epicurious', 0.7949345111846924),
 ('japanese', 0.793388843536377),
 ('cuisine', 0.7755702137947083),
 ('italian', 0.7742248177528381),
 ('ethnic', 0.7741429805755615),
 ('seriouseats_com', 0.7709793448448181)]

In [21]:
# we can call the vector of each word
model.wv['kettle']

array([ 0.01251994, -0.02555312, -0.00687722,  0.00985285,  0.0989612 ,
       -0.07703198,  0.02115107,  0.1904423 ,  0.08889545, -0.08537651,
        0.06746045,  0.00155182,  0.1027201 ,  0.05648083, -0.05895937,
       -0.19863664,  0.15604645, -0.07589505, -0.06632858, -0.08751618,
        0.09775511,  0.04701078,  0.05630622, -0.00256222, -0.04699665,
       -0.10184593, -0.03058753,  0.01046911, -0.08086667, -0.05634516,
       -0.08622472,  0.08457261, -0.13320242,  0.1105662 , -0.05405145,
        0.13396268,  0.09342658, -0.14760794, -0.03355089, -0.08638977,
       -0.05131674,  0.03127664,  0.0241367 , -0.03394893, -0.08779122,
        0.14100365, -0.02802504,  0.16016762, -0.01729349,  0.08059856,
        0.11174127, -0.01646334, -0.05063391,  0.00230364, -0.06663819,
        0.11572168,  0.20512606, -0.00102523, -0.04778711, -0.1137407 ,
       -0.07044202,  0.1035508 ,  0.03851005, -0.00163973,  0.01187798,
        0.09902862, -0.02715607,  0.15312704, -0.03289685, -0.17

In [22]:
model.wv.vectors.shape

(4857, 300)

In [ ]:
# from here you can ennter key-word dicts for mapping
model.wv.key_to_index

## Training Word2Vec from disk

Let's assume you want to train a word-embeddding model from disk. You downloaded all of Wikipedia or one of the large (multi GB datasets from Huggingface)

In [31]:
# open file (not read yet) from disk
texts_reddit = open('/content/reddit_r_cooking_sample.jsonl','r')

In [34]:
# read single line (this will iterate over the lines)
texts_reddit.readline()

'{"text":"My Pro 500 is going on 18 years old. Thing is a tank. Just don\'t drop it on your toe","meta":{"section":"Cooking","utc":1518485096}}\n'

In [35]:
# Decode JSON
json.loads(texts_reddit.readline())

{'text': 'deglazing works ok. but not as well as on a stainless pan. but yes heating and scrubbing with a nylon brush works pretty good too. also i use a wooden utensil.',
 'meta': {'section': 'Cooking', 'utc': '1413146528'}}

We need to turn our comments into sentences (tokenize) and preprocess. No need to do on-the-fly preprocessing 15 times
For that we create a new file `sentences.txt`, we tokenize our texts and write all sentences as lines into the new file. Using 1-sentence-per-line in TXTs is a common approach.

In [36]:
# We need re-open to start from top
texts_reddit = open('/content/reddit_r_cooking_sample.jsonl','r')

In [37]:
# open file
with open('sentances.txt','w') as f:
  for line in texts_reddit: # iterate over the json-lines with comments (alternative to readline())
    line = json.loads(line) # decode json
    for sent in sent_tokenize(line['text']): # sent-tokenize
      f.write(sent) # write sents into the new file
      f.write('\n')
  f.close()

The next step is not easy but important and your first step to writing "real code".
We need to define something that allows us to retrieve our sentences from the stored file one by one (and start from the beginning after the last one).

A class with an `__iter__` function can help here. This becomes an iterator that yields them one by one. `yield` is different from `return`. The latter ends an execution and returns the "overall" result of a function. `yield` is called repeatedly.

In [38]:
path = "/content/sentances.txt"

In [39]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""
    def __iter__(self):
        for line in open(path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

Let's try out how that works

In [40]:
# instantiate a corpus object
sentences_disk = MyCorpus()

In [41]:
# define a generator (similar to list comprehension but on "stand-by")
test_gen = (a for a in sentences_disk)

In [48]:
# every time we call next, it runs one iteration
next(test_gen)

['some',
 'of',
 'those',
 'grain',
 'sausages',
 'are',
 'really',
 'good',
 'and',
 'you',
 'can',
 'slice',
 'them']

Let's train our Phrases model from the disk-corpus

In [49]:
sentences_disk = MyCorpus()

In [50]:
phrase_model = Phrases(sentences_disk, min_count=25, threshold=20, connector_words=ENGLISH_CONNECTOR_WORDS)

2022-11-01 16:07:54,089 : INFO : collecting all words and their counts
2022-11-01 16:07:54,092 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-11-01 16:07:54,427 : INFO : PROGRESS: at sentence #10000, processed 123637 words and 74036 word types
2022-11-01 16:07:54,792 : INFO : PROGRESS: at sentence #20000, processed 245368 words and 129436 word types
2022-11-01 16:07:55,133 : INFO : collected 176735 token types (unigram + bigrams) from a corpus of 359868 words and 29445 sentences
2022-11-01 16:07:55,136 : INFO : merged Phrases<176735 vocab, min_count=25, threshold=20, max_vocab_size=40000000>
2022-11-01 16:07:55,139 : INFO : Phrases lifecycle event {'msg': 'built Phrases<176735 vocab, min_count=25, threshold=20, max_vocab_size=40000000> in 1.05s', 'datetime': '2022-11-01T16:07:55.139108', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [ ]:
for phrase, score in phrase_model.find_phrases(sentences_disk).items():
    print(phrase, score)

🚀🚀🚀
**Efficiency** is key when working from disk.
Let's preprocess the inputs using simple-prepro and the phrases model.
Since we preprocess our sentences into lists we need to store them using json such that we can load them into python objects, not strings

In [52]:
sentences_disk = MyCorpus()

In [53]:
# open new file (txt file with json-input)
with open('sentances_phrases.txt','w') as f:
  for sent in sentences_disk: # iterate over the json-lines with comments (alternative to readline())
    f.write(json.dumps(phrase_model[sent])) # write sents into the new file
    f.write('\n')
  f.close()

In [54]:
path = '/content/sentances_phrases.txt'

In [55]:
class MyCorpus_processed:
    """An iterator that yields sentences (lists of str)."""
    def __iter__(self):
        for line in open(path):
            # assume there's one document per line, tokens separated by whitespace
            yield json.loads(line)

In [56]:
sentences_disk = MyCorpus_processed()

In [57]:
# or we just add it to the training
model = Word2Vec(sentences=sentences_disk, 
                               vector_size=300, 
                               window=5, 
                               min_count=5, 
                               workers=4, 
                               epochs=15)

2022-11-01 16:12:31,547 : INFO : collecting all words and their counts
2022-11-01 16:12:31,564 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-01 16:12:31,689 : INFO : PROGRESS: at sentence #10000, processed 121743 words, keeping 9863 word types
2022-11-01 16:12:31,820 : INFO : PROGRESS: at sentence #20000, processed 241677 words, keeping 13807 word types
2022-11-01 16:12:31,961 : INFO : collected 16762 word types from a corpus of 354479 raw words and 29445 sentences
2022-11-01 16:12:31,970 : INFO : Creating a fresh vocabulary
2022-11-01 16:12:32,024 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 4857 unique words (28.98% of original 16762, drops 11905)', 'datetime': '2022-11-01T16:12:32.023991', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-11-01 16:12:32,027 : INFO : Word2Vec lifecycle event 

In [59]:
model.wv.most_similar('parsley')

[('thyme', 0.9535082578659058),
 ('oregano', 0.949043333530426),
 ('basil', 0.9457336068153381),
 ('coriander', 0.9394667148590088),
 ('scallions', 0.9386905431747437),
 ('salt_pepper', 0.933724582195282),
 ('ginger', 0.9317354559898376),
 ('rosemary', 0.9297377467155457),
 ('minced', 0.9290481209754944),
 ('cumin', 0.9265708923339844)]

### Bonus: Training FastText

training of FastText is syntax-wise the same.
There are a few other paras that you can tune

In [60]:
model_fasttext = FastText(sentences = sentences_disk, 
                          vector_size=300, 
                          window=8, 
                          min_count=5, 
                          workers=4, 
                          epochs=15)

2022-11-01 16:14:46,645 : INFO : collecting all words and their counts
2022-11-01 16:14:46,649 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-01 16:14:46,724 : INFO : PROGRESS: at sentence #10000, processed 121743 words, keeping 9863 word types
2022-11-01 16:14:46,794 : INFO : PROGRESS: at sentence #20000, processed 241677 words, keeping 13807 word types
2022-11-01 16:14:46,861 : INFO : collected 16762 word types from a corpus of 354479 raw words and 29445 sentences
2022-11-01 16:14:46,866 : INFO : Creating a fresh vocabulary
2022-11-01 16:14:46,899 : INFO : FastText lifecycle event {'msg': 'effective_min_count=5 retains 4857 unique words (28.98% of original 16762, drops 11905)', 'datetime': '2022-11-01T16:14:46.898992', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-11-01 16:14:46,901 : INFO : FastText lifecycle event 

In [62]:
model_fasttext.wv.most_similar('knife')

[('wife', 0.8343323469161987),
 ('life', 0.7492570877075195),
 ('hi', 0.710981011390686),
 ('kitchenaid', 0.7067800760269165),
 ('environment', 0.6972745060920715),
 ('sharp', 0.6842833161354065),
 ('sharpen', 0.6736289858818054),
 ('iron', 0.6505971550941467),
 ('profession', 0.6477628350257874),
 ('knives', 0.6448550820350647)]

In [63]:
model.wv['powder']

array([-5.51166274e-02,  5.88585377e-01,  1.34537190e-01,  9.33914423e-01,
        1.46900594e-01,  3.94622058e-01,  4.42739964e-01, -4.24239412e-03,
        3.70113790e-01,  2.35815242e-01, -3.90817434e-01, -2.70081982e-02,
        5.96169114e-01, -4.83595043e-01,  9.03829187e-02,  3.97176534e-01,
       -1.12596326e-01,  2.07230151e-01, -6.33924380e-02,  2.67278939e-01,
        1.22167774e-01,  5.55733144e-01, -2.37716556e-01,  3.42581809e-01,
        2.23291785e-01, -3.79723012e-01,  3.12910825e-01, -7.85174370e-01,
       -5.02312541e-01, -2.88786292e-01, -4.40250129e-01, -2.30905548e-01,
       -4.76674467e-01, -3.92863482e-01,  1.81958880e-02, -1.97329924e-01,
       -7.22720325e-02, -2.27187008e-01,  3.19248646e-01,  2.77612060e-01,
        5.50923467e-01,  6.59968913e-01, -4.59815890e-01,  1.37116745e-01,
       -3.07883695e-03, -4.06670384e-02, -1.37955789e-02,  4.45950627e-01,
        3.25371712e-01,  1.26919985e-01, -1.41771212e-01, -3.95996094e-01,
       -5.44713259e-01, -

## Visualizing Word-Vectors

now that we have our Word-vectors we should be able to reduce their dimensionality to explore visually

In [64]:
!pip install umap-learn -q

     |████████████████████████████████| 88 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 


In [65]:
import random
import umap
import altair as alt

In [66]:
# picking 2000 random vectors from the W2V model
idx = random.sample(range(len(model.wv.vectors)), 2000)

In [70]:
# creating 2D reduction
umap_reducer = umap.UMAP(random_state=42, n_components=2)
embeddings = umap_reducer.fit_transform(model.wv.vectors[idx])

In [71]:
# df for plot
df_plot = pd.DataFrame(embeddings, columns=['x','y'])

In [72]:
# vector-labels
labels = [model.wv.index_to_key[ix] for ix in idx]

In [73]:
df_plot['labels'] = labels

In [74]:
# plot
alt.Chart(df_plot).mark_circle(size=60).encode(
    x='x',
    y='y',
    tooltip=['labels']
).properties(
    width=800,
    height=600
).interactive()

alt.Chart(...)

## Create sentence embeddings from our W2V model

The final aim is to use the custom W2V embeddings to vectorize sentences
We will look at average vectors and tfidf weighted avg. embeddings

In [75]:
test_sents = ['I love chicken super much with soy',
              'I enjoy asian food, especially chicken',
              'Give me cake', 'mexican food is amazing', 
              'I enjoy cuisine italian']

### Average W2V vectors

In [76]:
# tokenize
tokens = phrase_model[utils.simple_preprocess(test_sents[0])]

In [78]:
# filter out only those words that are part of the vocab
tokens = [t for t in tokens if t in model.wv.key_to_index.keys()]

In [82]:
# create average-vectors
avg_vec = np.average([model.wv[t] for t in tokens], axis=0)

let's package this process up into a vectorizer-function

In [84]:
def w2v_vectorize(text):
  tokens = phrase_model[utils.simple_preprocess(text)] # preprocess just as model inputs
  tokens = [t for t in tokens if t in model.wv.key_to_index.keys()] # filter only tokens that are in vocab
  return np.average([model.wv[t] for t in tokens], axis=0) # calculate avg vector

In [85]:
# it's a goof idea to stack them using numpy into a matrix
vecs = np.vstack([w2v_vectorize(s) for s in test_sents])

In [86]:
# quick explaininng of the vectors (not really part of the code)
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vecs)

array([[0.99999964, 0.34846756, 0.13731556, 0.26950514, 0.34354964],
       [0.34846756, 0.9999999 , 0.19795777, 0.68343115, 0.6109475 ],
       [0.13731556, 0.19795777, 1.0000002 , 0.216211  , 0.28588519],
       [0.26950514, 0.68343115, 0.216211  , 0.99999994, 0.7157878 ],
       [0.34354964, 0.6109475 , 0.28588519, 0.7157878 , 0.9999995 ]],
      dtype=float32)

### TFIDF weighted W2V Embeddings

Very similar to avg-embeddings, however here we will use sklearn TfidfVectorizer (that one we already know) to weight our vecs
The approach is a bit "hacky" but efficient

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
# function that does absolutely nothing...
# cause we do prepro and tokenization in one using gensim, we will define it for prepro
def dummy_fun(doc):
    return doc

In [89]:
[phrase_model[utils.simple_preprocess(text)] for text in test_sents]

[['love', 'chicken', 'super', 'much', 'with', 'soy'],
 ['enjoy', 'asian', 'food', 'especially', 'chicken'],
 ['give', 'me', 'cake'],
 ['mexican', 'food', 'is', 'amazing'],
 ['enjoy', 'cuisine', 'italian']]

In [90]:
# we define a preprocessing function to pass into the TfidfVectorizer
def gensim_prepro(doc):
  return phrase_model[utils.simple_preprocess(doc)]

In [93]:
# we turn of any preprocessing and align vocabulary with the one
# used by our embeddings
# that will allow us to use TFIDF vectors to weight the embeddings

tfidf_new_text = TfidfVectorizer(
    vocabulary=model.wv.key_to_index.keys(), # here using the W2V vocab
    tokenizer=dummy_fun,
    preprocessor=gensim_prepro,
    token_pattern=None)  

In [94]:
# create TFIDF matrix (we could also just use that one for search)
new_tfidf = tfidf_new_text.fit_transform(test_sents)

In [95]:
new_tfidf

<5x4857 sparse matrix of type '<class 'numpy.float64'>'
	with 21 stored elements in Compressed Sparse Row format>

This here is a cool little trick: Since N-columns for the TFIDF is the same as n-rows for our word-embeddings we can simply take a dot-product here.
Another cool feature: this can be done sequentially for large datasets (when no space in ram)

In [96]:
# calculating TFIDF-weighted avg. embeddings
test_w2v_tfidf = new_tfidf @ model.wv.vectors

In [98]:
test_w2v_tfidf.shape

(5, 300)

In [99]:
cosine_similarity(test_w2v_tfidf)

array([[1.        , 0.29710012, 0.14867027, 0.31735163, 0.34757172],
       [0.29710012, 1.        , 0.18192084, 0.66205786, 0.63341578],
       [0.14867027, 0.18192084, 1.        , 0.20667902, 0.24786758],
       [0.31735163, 0.66205786, 0.20667902, 1.        , 0.7374783 ],
       [0.34757172, 0.63341578, 0.24786758, 0.7374783 , 1.        ]])

## Using these embeddings for semantic search
We can use such embeddings (and others) for semantic search (similarity maximization) and also downstream in unsuprvised/supervised tasks.

In [100]:
# create TFIDF matrix for all
tfidf_all = tfidf_new_text.fit_transform(data['text'])

In [102]:
# get vecs by dot-product
tfidf_w2v_all = tfidf_all @ model.wv.vectors

In [114]:
# make query and transform it into same vector-space

query = 'Italian breakfast'

tfidf_q = tfidf_new_text.transform([query]) 
tfidf_w2v_q = tfidf_q @ model.wv.vectors

In [115]:
# calculate cos-sim between the query and all vecs

distances = cosine_similarity(tfidf_w2v_q,tfidf_w2v_all)

In [116]:
# get corresponding texts
ids = np.flip(np.argsort(distances))[0]
ids

array([8264, 8693, 9795, ..., 8114,  745, 5984])

In [117]:
# print
for ix in ids[:10]:
  print(data['text'].values[ix])

Arizona: breakfast burritos.
Italian struffoli !
"mexican" lasagna. mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
I might have a few for you. Probably nothing Mediterranean though. https://www.copymethat.com/r/hzvRQdG/after-school-antipasto-pinwheel-sandwich/ https://www.copymethat.com/r/bRnMfB2/american-italian-pasta-salad-ar-carol-em/ https://www.copymethat.com/r/T5TGZCE/greek-orzo-salad-ar-patrice/ https://www.copymethat.com/r/GmSceZb/ms-easy-tex-mex-vegan-salad/
This is really good: http://cooking.nytimes.com/recipes/1017946-baked-cheesy-pasta-casserole-with-wild-mushrooms
Here are some of my favorite (Not super common recipes) 1. Pasta alla Norcina. A delicious cheesy, sausage pasta dish. https://www.the-pasta-project.com/pasta-alla-norcina-sausage-pasta-recipe-from-umbria/ 2. Shrimp Fra Diavolo. A spicy shrimp pasta dish. https://www.allrecipes.com/recipe/238843/chef-johns-shrimp-fra-diavolo/ 3. Spicy Brussell Sprouts. So delicious, just had them. https://food52.com/blog/4857-m

### Serialization

Gensim models can be (ans should be) saved to disk after training.

In [118]:
phrase_model.save('bigram_model.m')

2022-11-01 16:45:18,220 : INFO : Phrases lifecycle event {'fname_or_handle': 'bigram_model.m', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-11-01T16:45:18.220589', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-11-01 16:45:18,519 : INFO : saved bigram_model.m


In [119]:
model.save('w2v_food.m')

2022-11-01 16:45:18,678 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'w2v_food.m', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-11-01T16:45:18.678195', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-11-01 16:45:18,690 : INFO : not storing attribute cum_table
2022-11-01 16:45:18,778 : INFO : saved w2v_food.m


In [121]:
g = Word2Vec.load('/content/w2v_food.m')

2022-11-01 16:46:13,635 : INFO : loading Word2Vec object from /content/w2v_food.m
2022-11-01 16:46:13,682 : INFO : loading wv recursively from /content/w2v_food.m.wv.* with mmap=None
2022-11-01 16:46:13,684 : INFO : setting ignored attribute cum_table to None
2022-11-01 16:46:13,795 : INFO : Word2Vec lifecycle event {'fname': '/content/w2v_food.m', 'datetime': '2022-11-01T16:46:13.795158', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [122]:
g.wv.most_similar('garlic')

[('unpeeled', 0.7983607649803162),
 ('celery', 0.7926087379455566),
 ('cloves', 0.7867130041122437),
 ('onions', 0.776432454586029),
 ('chopped', 0.7641382217407227),
 ('minced', 0.763360321521759),
 ('ginger', 0.7594746947288513),
 ('carrots', 0.7548808455467224),
 ('finely', 0.7508522868156433),
 ('olive_oil', 0.7437301874160767)]